#Treinar a rede GAN para gerar novos digitos


Uma Rede Generativa Adversária (GAN) é um tipo de modelo de aprendizado de máquina que é capaz de gerar novos dados que se parecem com os dados de treinamento. Isso é feito treinando dois modelos de rede neural separados, um chamado gerador e outro chamado discriminador, para trabalharem juntos.

O gerador tenta criar novos dados que se pareçam o mais possível com os dados de treinamento, enquanto o discriminador tenta diferenciar os dados gerados pelo gerador dos dados de treinamento reais. O gerador é treinado para tentar enganar o discriminador, enquanto o discriminador é treinado para ficar cada vez melhor em distinguir os dados gerados dos dados reais.

Desta forma, o gerador aprende a criar novos dados que são tão bons quanto possível, enquanto o discriminador aprende a identificar os dados gerados. É uma abordagem interessante para gerar novos dados que podem ser úteis em várias aplicações, como gerar imagens realistas a partir de amostras de treinamento ou criar novas músicas com base em exemplos de músicas existentes.

In [1]:
#!pip uninstall tensorflow
#!pip install tensorflow

#!pip uninstall keras
#!pip install keras

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

#from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.layers import Reshape, Dense, Dropout, Flatten
from tensorflow.keras.layers import Convolution2D, UpSampling2D
from tensorflow.keras.optimizers.legacy import Adam

from keras.layers import Input
from keras.models import Model, Sequential
#from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import LeakyReLU
#from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers import BatchNormalization
from keras.datasets import mnist
#from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

os.environ["KERAS_BACKEND"] = "tensorflow"

O conjunto de dados de treinamento foi normalizado para valores entre **-1 e 1**. Esse ajuste é necessário para utilizar a função de ativação **Tangente Hiperbólica** (*tanh*) da ultima camada da rede geradora.

In [3]:
np.random.seed(1000)

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train.reshape(60000, 784)

adam = Adam(learning_rate=0.0002, beta_1=0.5)

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
randomDim = 100
adam = Adam(learning_rate=0.0002, beta_1=0.5)

In [5]:
generator = Sequential()
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Dense(512))
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784, activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [6]:
discriminator = Sequential()
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(512))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [7]:
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
x = generator(ganInput)
ganOutput = discriminator(x)
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

dLosses = []
gLosses = []

In [8]:
def plotLoss():
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig("/content/drive/MyDrive/GAN_Mack24/Funcao_Perda.png")


In [9]:
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/content/drive/MyDrive/GAN_Mack24/digito_gerado_%d.png' % epoch)

In [10]:
def saveModels():
    generator.save('/content/drive/MyDrive/GAN_Mack24/gerador.h5')
    discriminator.save('/content/drive/MyDrive/GAN_Mack24/discriminador.h5')

O treino do modelo é feito em um *loop* aninhado. O primeiro vai iterar com o total de épocas definidos por parâmetro do usuário, e em cada época é feito um novo *loop* que é calculado a partir da quantidade da amostra dividido pelo tamanho do *batch*. Se o valor do *batch* for baixo vai gerar uma iteração interna com mais repetições internas, por outro lado se o *batch* for alto, vai gerar menos iteração no *loop* interno.
O tamanho do *batch* também impacta no tamanho do ruído que será inserido na imagem original para que a rede possa ser treinada. Se existir muito ruído, a rede irá demorar mais para convergir e se o ruído for baixo, a rede não terá generalização suficiente para ser efetiva.

In [11]:
def train(epochs=1, batchSize=128):
    batchCount = int(X_train.shape[0] / batchSize)
    print('Épocas:', epochs)
    print('Batch size:', batchSize)
    print('Batch por épocas:', batchCount)

    for e in range(1, epochs+1):
        print('-'*15, 'Época %d' % e, '-'*15)
        for _ in  tqdm(range(batchCount)):
            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]

            generatedImages = generator.predict(noise, verbose=0)
            X = np.concatenate([imageBatch, generatedImages])

            yDis = np.zeros(2*batchSize)
            yDis[:batchSize] = 0.9

            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, yDis)

            noise = np.random.normal(0, 1, size=[batchSize, randomDim])
            yGen = np.ones(batchSize)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, yGen)

        dLosses.append(dloss)
        gLosses.append(gloss)

        plotGeneratedImages(e)

In [ ]:
train(100, 128)

Épocas: 100
Batch size: 128
Batch por épocas: 468
--------------- Época 1 ---------------


100%|██████████| 468/468 [01:19<00:00,  5.88it/s]


4/4 [==============================] - 0s 4ms/step
--------------- Época 2 ---------------


100%|██████████| 468/468 [01:08<00:00,  6.85it/s]

4/4 [==============================] - 0s 3ms/step


...

1/4 [======>.......................] - ETA: 0s

4/4 [==============================] - 0s 4ms/step
--------------- Época 11 ---------------


100%|██████████| 468/468 [01:07<00:00,  6.98it/s]

1/4 [======>.......................] - ETA: 0s

4/4 [==============================] - 0s 4ms/step


In [ ]:
plotLoss()

In [ ]:
saveModels()

In [ ]:
def gerar_digito(n_ex=1, dim=(1, 10), figsize=(12, 2)):
    noise = np.random.normal(0, 1, size=(n_ex, randomDim))
    imagem_gerada = generator.predict(noise)
    imagem_gerada = imagem_gerada.reshape(28, 28)

    plt.imshow(imagem_gerada, interpolation='nearest', cmap='gray_r')
    plt.figure(figsize=figsize)
    plt.tight_layout()
    plt.show()

In [ ]:
gerar_digito()

##Referências

Código adaptado de https://github.com/Zackory/Keras-MNIST-GAN/blob/master/mnist_gan.py

Goodfellow, Ian, et al. "Generative adversarial nets." Advances in neural information processing systems. 2014.

Radford, Alec, et al. "Unsupervised representation learning with deep convolutional generative adversarial networks." arXiv preprint arXiv:1511.06434 (2015).
